In [ ]:
import pandas as pd
from thefuzz import process

In [2]:
tree_data = pd.read_excel('tree_data.xlsx')
la_trees_fire_temp = pd.read_csv('la_trees_fire_temp.csv')

In [5]:
def match_tree_name(name, choices, threshold=80):
    """Fuzzy match a name to a list of choices with a threshold."""
    if pd.isna(name):
        return None
    match, score = process.extractOne(name, choices)
    return match if score >= threshold else None

def match_treegroup_name(name, choices, threshold=65):
    """Fuzzy match a TreeGroup name to a list of choices with a lower threshold."""
    if pd.isna(name):
        return None
    name = name.lower().strip()
    choices = [c.lower().strip() for c in choices]
    match, score = process.extractOne(name, choices)
    return match if score >= threshold else None

In [6]:
# Get unique tree names from tree_data
tree_names = tree_data['Common Name'].unique()

# Match each name in la_trees_fire_temp to the closest in tree_data
la_trees_fire_temp['Matched Tree Name'] = la_trees_fire_temp['Species'].apply(
    lambda x: match_tree_name(x, tree_names)
)


In [7]:

# Merge Firewise Rating using the matched names
merged = la_trees_fire_temp.merge(
    tree_data[['Common Name', 'Firewise Rating']],
    left_on='Matched Tree Name',
    right_on='Common Name',
    how='left'
)

In [8]:
merged

,Latitude,Longitude,Species,DBH,Any_Protected,tree_id,geometry,index_right,OBJECTID,YEAR_,...,Shape__Area,Shape__Length,tree_appear_count,FIRE_SIZE_CATEGORY,USDA_FIRE_SIZE_CLASS,TreeGroup,Annual_Temp,Matched Tree Name,Common Name,Firewise Rating
0,34.154814,-118.589896,coast live oak (Quercus agrifolia),25.0,False,398,POINT (-118.58989569441763 34.15481399358518),18162,18164,1944.0,...,2.694018e+07,35271.399924,1,Large,F - Large,Oak,62.2,live oak,live oak,AT RISK Firewise (3)
1,34.154815,-118.589929,coast live oak (Quercus agrifolia),13.0,False,399,POINT (-118.5899292220234 34.154815103381026),18162,18164,1944.0,...,2.694018e+07,35271.399924,1,Large,F - Large,Oak,62.2,live oak,live oak,AT RISK Firewise (3)
2,34.275480,-118.541392,MyLA311 Added (MyLA311 Added),0.0,False,782,POINT (-118.54139204162942 34.27547960352691),15796,15798,1959.0,...,3.709861e+06,8002.545419,2,Medium,E - Medium,Myla311,67.3,None,NaN,NaN
3,34.145944,-118.505713,MyLA311 Added (MyLA311 Added),0.0,False,1620,POINT (-118.50571275517476 34.14594381558009),15672,15674,1960.0,...,4.783674e+05,4025.625330,1,Small,C - Small,Myla311,65.6,None,NaN,NaN
4,34.089382,-118.452487,coast live oak (Quercus agrifolia),10.0,False,1826,POINT (-118.4524866762754 34.089381786916896),15424,15426,1961.0,...,3.640036e+07,48599.914373,1,Large,G - Large,Oak,65.3,live oak,live oak,AT RISK Firewise (3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51920,34.281985,-118.524873,swamp mallee (Eucalyptus spathulata),1.0,False,848183,POINT (-118.52487262848 34.281985493955),17825,17827,1947.0,...,2.472436e+06,7602.760800,1,Medium,E - Medium,Eucalyptus,64.1,eucalyptus,eucalyptus,NOT Firewise (4)
51921,34.281909,-118.524855,cork oak (Quercus suber),1.0,False,848184,POINT (-118.52485539633 34.281909200602),17825,17827,1947.0,...,2.472436e+06,7602.760800,1,Medium,E - Medium,Oak,64.1,black oak,black oak,Firewise (1)
51922,34.281829,-118.524832,cork oak (Quercus suber),1.0,False,848185,POINT (-118.524831927001 34.281829415729),17825,17827,1947.0,...,2.472436e+06,7602.760800,1,Medium,E - Medium,Oak,64.1,black oak,black oak,Firewise (1)
51923,34.281753,-118.524804,cork oak (Quercus suber),1.0,False,848186,POINT (-118.524804103646 34.281752631767),17825,17827,1947.0,...,2.472436e+06,7602.760800,1,Medium,E - Medium,Oak,64.1,black oak,black oak,Firewise (1)


In [9]:
# Find rows where Firewise Rating is NaN
nan_rows = merged[merged['Firewise Rating'].isna()].copy()

# Try to match TreeGroup to Common Name for these rows (lower threshold)
nan_rows['Matched TreeGroup Name'] = nan_rows['TreeGroup'].apply(
    lambda x: match_treegroup_name(x, tree_names, threshold=65)
)

# Merge again to get Firewise Rating using the new matches
nan_rows = nan_rows.merge(
    tree_data[['Common Name', 'Firewise Rating']],
    left_on='Matched TreeGroup Name',
    right_on='Common Name',
    how='left',
    suffixes=('', '_TreeGroup')
)

In [10]:
# Update merged Firewise Rating where possible
for idx, row in nan_rows.iterrows():
    if pd.notna(row['Firewise Rating_TreeGroup']):
        merged.loc[idx, 'Firewise Rating'] = row['Firewise Rating_TreeGroup']

In [11]:
# Show unique TreeGroup values where Firewise Rating is still NaN
unmatched_treegroups = merged[merged['Firewise Rating'].isna()]['TreeGroup'].unique()
print(unmatched_treegroups)

['Myla311' 'Jacaranda' 'Cupaniopsis' 'Bignoniaceae' 'Psidium'
 'Lophostemon' 'Persea' 'Fig' 'Melaleuca' 'Ceratonia' 'Heteromeles'
 'Pithecellobium' 'Tipuana' 'Acacia' 'Erythrina' 'Prunus' 'Eucalyptus'
 'Parkinsonia' 'Xylosma' 'Citrus' 'Lyonothamnus' 'Koelreuteria'
 'Araucaria']


In [12]:
# Assign Firewise Rating for TreeGroup 'Persea' where missing
merged.loc[
    (merged['TreeGroup'] == 'Persea') & (merged['Firewise Rating'].isna()),
    'Firewise Rating'
] = 'AT RISK Firewise (3)'

In [13]:
# Assign Firewise Rating for TreeGroup 'Fig' where missing
merged.loc[
    (merged['TreeGroup'] == 'Fig') & (merged['Firewise Rating'].isna()),
    'Firewise Rating'
] = 'MODERATELY Firewise (2)'

In [14]:
# Assign Firewise Rating for TreeGroup 'Acacia' where missing
merged.loc[
    (merged['TreeGroup'] == 'Acacia') & (merged['Firewise Rating'].isna()),
    'Firewise Rating'
] = 'MODERATELY Firewise (2)'

In [15]:
# Assign Firewise Rating for TreeGroup 'Prunus' where missing
merged.loc[
    (merged['TreeGroup'] == 'Prunus') & (merged['Firewise Rating'].isna()),
    'Firewise Rating'
] = 'MODERATELY Firewise (2)'

In [16]:
# Assign Firewise Rating for TreeGroup 'Eucalyptus' where missing
merged.loc[
    (merged['TreeGroup'] == 'Eucalyptus') & (merged['Firewise Rating'].isna()),
    'Firewise Rating'
] = 'NOT Firewise (4)'

In [17]:
# Assign Firewise Rating for TreeGroup 'Citrus' where missing
merged.loc[
    (merged['TreeGroup'] == 'Citrus') & (merged['Firewise Rating'].isna()),
    'Firewise Rating'
] = 'AT RISK Firewise (3)'

In [18]:
# Assign Firewise Rating for TreeGroup 'Koelreuteria' where missing
merged.loc[
    (merged['TreeGroup'] == 'Koelreuteria') & (merged['Firewise Rating'].isna()),
    'Firewise Rating'
] = 'Firewise (1)'

In [19]:
merged[merged['Firewise Rating'].isna()]

,Latitude,Longitude,Species,DBH,Any_Protected,tree_id,geometry,index_right,OBJECTID,YEAR_,...,Shape__Area,Shape__Length,tree_appear_count,FIRE_SIZE_CATEGORY,USDA_FIRE_SIZE_CLASS,TreeGroup,Annual_Temp,Matched Tree Name,Common Name,Firewise Rating
2,34.275480,-118.541392,MyLA311 Added (MyLA311 Added),0.0,False,782,POINT (-118.54139204162942 34.27547960352691),15796,15798,1959.0,...,3.709861e+06,8002.545419,2,Medium,E - Medium,Myla311,67.3,None,NaN,NaN
3,34.145944,-118.505713,MyLA311 Added (MyLA311 Added),0.0,False,1620,POINT (-118.50571275517476 34.14594381558009),15672,15674,1960.0,...,4.783674e+05,4025.625330,1,Small,C - Small,Myla311,65.6,None,NaN,NaN
7,34.132728,-118.478920,jacaranda (Jacaranda mimosifolia),11.0,False,2066,POINT (-118.478919571 34.1327277743),16505,16507,1955.0,...,7.001529e+05,6152.939792,1,Medium,D - Medium,Jacaranda,63.7,None,NaN,NaN
9,34.132561,-118.478953,jacaranda (Jacaranda mimosifolia),22.0,False,2068,POINT (-118.478953028 34.1325613221),16505,16507,1955.0,...,7.001529e+05,6152.939792,1,Medium,D - Medium,Jacaranda,63.7,None,NaN,NaN
52,34.139447,-118.527297,carrotwood (Cupaniopsis anacardioides),3.0,False,3962,POINT (-118.527297157 34.1394474078),18162,18164,1944.0,...,2.694018e+07,35271.399924,1,Large,F - Large,Cupaniopsis,62.2,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51694,34.288939,-118.556508,jacaranda (Jacaranda mimosifolia),5.0,False,841405,POINT (-118.556508301 34.288939292),17346,17348,1950.0,...,2.516131e+07,31893.552189,1,Large,F - Large,Jacaranda,63.8,None,NaN,NaN
51695,34.288917,-118.556413,jacaranda (Jacaranda mimosifolia),8.0,False,841406,POINT (-118.556413467 34.2889169056),17346,17348,1950.0,...,2.516131e+07,31893.552189,1,Large,F - Large,Jacaranda,63.8,None,NaN,NaN
51696,34.288815,-118.556424,jacaranda (Jacaranda mimosifolia),5.0,False,841407,POINT (-118.556424004 34.2888149228),17346,17348,1950.0,...,2.516131e+07,31893.552189,1,Large,F - Large,Jacaranda,63.8,None,NaN,NaN
51711,34.289441,-118.556775,jacaranda (Jacaranda mimosifolia),10.0,False,841424,POINT (-118.556774908 34.2894405248),17346,17348,1950.0,...,2.516131e+07,31893.552189,3,Large,F - Large,Jacaranda,63.8,None,NaN,NaN


In [21]:
# Drop the columns 'Matched Tree Name' and 'Common Name'
merged = merged.drop(columns=['Matched Tree Name', 'Common Name'])

In [23]:
merged

,Latitude,Longitude,Species,DBH,Any_Protected,tree_id,geometry,index_right,OBJECTID,YEAR_,...,GIS_ACRES,DECADES,Shape__Area,Shape__Length,tree_appear_count,FIRE_SIZE_CATEGORY,USDA_FIRE_SIZE_CLASS,TreeGroup,Annual_Temp,Firewise Rating
0,34.154814,-118.589896,coast live oak (Quercus agrifolia),25.0,False,398,POINT (-118.58989569441763 34.15481399358518),18162,18164,1944.0,...,4548.5230,1950-1959,2.694018e+07,35271.399924,1,Large,F - Large,Oak,62.2,AT RISK Firewise (3)
1,34.154815,-118.589929,coast live oak (Quercus agrifolia),13.0,False,399,POINT (-118.5899292220234 34.154815103381026),18162,18164,1944.0,...,4548.5230,1950-1959,2.694018e+07,35271.399924,1,Large,F - Large,Oak,62.2,AT RISK Firewise (3)
2,34.275480,-118.541392,MyLA311 Added (MyLA311 Added),0.0,False,782,POINT (-118.54139204162942 34.27547960352691),15796,15798,1959.0,...,624.3035,1950-1959,3.709861e+06,8002.545419,2,Medium,E - Medium,Myla311,67.3,NaN
3,34.145944,-118.505713,MyLA311 Added (MyLA311 Added),0.0,False,1620,POINT (-118.50571275517476 34.14594381558009),15672,15674,1960.0,...,80.7574,1960-1969,4.783674e+05,4025.625330,1,Small,C - Small,Myla311,65.6,NaN
4,34.089382,-118.452487,coast live oak (Quercus agrifolia),10.0,False,1826,POINT (-118.4524866762754 34.089381786916896),15424,15426,1961.0,...,6151.9790,1960-1969,3.640036e+07,48599.914373,1,Large,G - Large,Oak,65.3,AT RISK Firewise (3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51920,34.281985,-118.524873,swamp mallee (Eucalyptus spathulata),1.0,False,848183,POINT (-118.52487262848 34.281985493955),17825,17827,1947.0,...,416.0731,1950-1959,2.472436e+06,7602.760800,1,Medium,E - Medium,Eucalyptus,64.1,NOT Firewise (4)
51921,34.281909,-118.524855,cork oak (Quercus suber),1.0,False,848184,POINT (-118.52485539633 34.281909200602),17825,17827,1947.0,...,416.0731,1950-1959,2.472436e+06,7602.760800,1,Medium,E - Medium,Oak,64.1,Firewise (1)
51922,34.281829,-118.524832,cork oak (Quercus suber),1.0,False,848185,POINT (-118.524831927001 34.281829415729),17825,17827,1947.0,...,416.0731,1950-1959,2.472436e+06,7602.760800,1,Medium,E - Medium,Oak,64.1,Firewise (1)
51923,34.281753,-118.524804,cork oak (Quercus suber),1.0,False,848186,POINT (-118.524804103646 34.281752631767),17825,17827,1947.0,...,416.0731,1950-1959,2.472436e+06,7602.760800,1,Medium,E - Medium,Oak,64.1,Firewise (1)


In [24]:
merged.to_csv('la_fire_tree_temp_score.csv', index=False)